In [125]:

import requests
import sqlite3
from requests.auth import HTTPBasicAuth
import pandas as pd
from sodapy import Socrata

In [126]:
# api endpoint
api_url = "https://data.colorado.gov/resource/4ykn-tg5h.json"
app_token = "0NT6cCdGg6VVAj2pRLjmAqDFu"
username = "glissmann.mark@gmail.com"
password = "K2PRH@mUDwgbfsV"
entityid = "18881009142"
api_key = "gxc0jsptwdopcqxffpl21c8b"
api_secret_key = "2w9yp2sqyp3d7nxkidj8v65e6ow7mos3e7bnuc64rqaiccyc4c" 


In [127]:
batch_size = 1000 
offset = 0

In [128]:
# connect to SQLite database (or create it)
conn = sqlite3.connect('business_entities.db')
cursor = conn.cursor()

In [132]:
# Fetch one record to determine the structure of the data
response = requests.get(api_url, params={'$limit': 1})
if response.status_code == 200:
    sample_data = response.json()
    if sample_data:
        columns = sample_data[0].keys()
        # Create table based on the keys in the data
        cursor.execute(f'''
        CREATE TABLE IF NOT EXISTS data (
            {', '.join([f'{col} TEXT' for col in columns])}
        )
        ''')
        conn.commit()
else:
    print(f"Failed to retrieve data: {response.status_code}")
    exit()

In [134]:
while True:
    # fetch a batch of data
    params = {
        '$limit': batch_size,
        '$offset': offset
    }

    response = requests.get(api_url, params=params)

    if response.status_code == 200:
        data = response.json()

        if not data:
            # no more data to fetch
            break
        
        # prepare insert statement
        placeholders = ', '.join(['?'] * len(columns))
        insert_statement = f'''
        INSERT OR IGNORE INTO colorado ({', '.join(columns)})
        VALUES ({placeholders})
        '''

        # process and insert data into the database
        for item in data:
            values = tuple(item.get(col) for col in columns)
            cursor.execute(insert_statement, values)
        
        # commit changes to the database
        conn.commit()

        # increment offset for the next batch
        offset += batch_size
    else:
        print(f"Failed to retreive data: {response.status_code}")
        break 

# close the database connection
conn.close()

In [121]:

# Convert the list of all records to a pandas DataFrame
df = pd.DataFrame(all_data)
print(f'Total records retrieved: {len(df)}')

# memory usage of the dataframe
memory = df.memory_usage(deep=True).sum()
print(f'Total memory usage: {memory} bytes or {round(memory/1073741824, 2)} gigabytes')

Total records retrieved: 120000
Total memory usage: 217838949 bytes or 0.2 gigabytes


In [122]:
print(f'Min entityformdate: {min(df.entityformdate)}')
print(f'Max entityformdate: {max(df.entityformdate)} \n')
print(df.head())

Min entityformdate: 2023-01-01T00:00:00.000
Max entityformdate: 2023-07-25T00:00:00.000 

      entityid                   entityname    principaladdress1  \
0  20221952116               Pommeander LLC     5295 Eldridge St   
1  20221960515  CC Management Services Inc.      319 Aabc Unit N   
2  20221963658        OliveBlack Events LLC  11675 Pink Phlox Dr   
3  20221969060     Three Hive Strategy, LLC    10127 Phillips Rd   
4  20221973469             The Nest of Hope    18608 E Vassar Dr   

  principalcity principalstate principalzipcode principalcountry  \
0        Arvada             CO            80002               US   
1         Aspen             CO            81611               US   
2        Parker             CO            80134               US   
3     Lafayette             CO            80026               US   
4        Aurora             CO            80013               US   

    entitystatus jurisdictonofformation                 entitytype  ...  \
0  Good Standing 